## Using SQLite for a data warehouse

## The proposal
- Data Analysis Requires Many Transformations
- These Are Often applied in a Layered Approach
- Create New Columns for Analysis and Model Features
- Stratify and Prepare Data for Model Training
- Use SQLite to Save the Transformed Data
- Like your own Local Data Warehouse
- Step by Step Notebook with Code Included

### 1. Load the SQLite Package

In [93]:
import sqlite3
import warnings
warnings.filterwarnings('ignore')
print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.39.3


### 2.Connect to the database

In [94]:
slconn = sqlite3.connect('aw_eda') # permanent database

In [95]:
print(slconn)

### 3. Create a cursor to execute statement to SQLite

In [96]:
cursor = slconn.cursor()

In [97]:
cursor

### Get Data from database management system...

**Load our simple SQLDB class from dbutils module...**

In [98]:
from dbutils.dbaccess import SQLDB

**Define the SQL server name**

In [99]:
sqlservername = 'DESKTOP-J8EP77U\TABULAR_2019'

**Create a SQL query to execute**
### Create the SQLDB object pointing to the SQL server
- we only need to overrride the constructor parameters that differ from what we need.
- we use the default of inegrated security.
- Parameters: Server, Integrated Security(True/False, Driver, Database)

In [100]:
aw_sql_is = SQLDB(sqlservername, True, '{ODBC Driver 17 for SQL Server}', 'AdventureWorksDW2017')

In [101]:
aw_sql_is.get_db_data('select top 2 * from dbo.DimProduct')

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,...,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
0,1,AR-5381,None,None,None,Adjustable Race,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current
1,2,BA-8327,None,None,None,Bearing Ball,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current


**Or store the query results to a dataframe**

In [102]:
dfproduct = aw_sql_is.get_db_data('select top 5 * from DimProduct')
dfproduct.head()

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,...,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
0,1,AR-5381,None,None,None,Adjustable Race,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current
1,2,BA-8327,None,None,None,Bearing Ball,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current
2,3,BE-2349,None,None,None,BB Ball Bearing,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current
3,4,BE-2908,None,None,None,Headset Ball Bearings,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current
4,5,BL-2036,None,None,None,Blade,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current


### Create another SQLDB object that uses SQL Server Security.

In [103]:
aw_sql_sqlsec = SQLDB(sqlservername, 
               False, '{ODBC Driver 17 for SQL Server}',
               'AdventureWorksDW2017',
               'user',
               '')

In [104]:
aw_sql_sqlsec.get_db_data('select top 2 * from dbo.DimProduct').head(3)

,ProductKey,ProductAlternateKey,ProductSubcategoryKey,WeightUnitMeasureCode,SizeUnitMeasureCode,EnglishProductName,SpanishProductName,FrenchProductName,StandardCost,FinishedGoodsFlag,...,ChineseDescription,ArabicDescription,HebrewDescription,ThaiDescription,GermanDescription,JapaneseDescription,TurkishDescription,StartDate,EndDate,Status
0,1,AR-5381,None,None,None,Adjustable Race,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current
1,2,BA-8327,None,None,None,Bearing Ball,,,None,False,...,None,None,None,None,None,None,None,2003-07-01,None,Current


### We can create a SQLDB object that points to any type of database with an ODBC driver. Below we connect to a PostgreSQL database.

In [105]:
aw_postgresdb = SQLDB('localhost', False, '{PostgreSQL ODBC Driver(UNICODE)}', 'northwind', 'postgres', 'XXXXXXXX')

In [106]:
aw_postgresdb.get_db_data('select * from customers limit 2')

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745


### Extract data from the backend databases, and do some data munging to add value.  

In [107]:
sql = '''
SELECT EnglishProductCategoryName as Category,
       sc.EnglishProductSubcategoryName as SubCat,
       p.EnglishProductName as Product,
       SUM(SalesAmount) as SalesAmount
FROM dbo.FactInternetSales          s
JOIN  dbo.DimProduct                p
  on (s.ProductKey = p.ProductKey) 
JOIN dbo.DimProductSubcategory      sc
  ON (p.ProductSubcategoryKey = sc.ProductSubcategoryKey)
JOIN dbo.DimProductCategory         pc
  ON (sc.ProductCategoryKey = pc.ProductCategoryKey)
GROUP BY EnglishProductCategoryName, 
         EnglishProductSubcategoryName,
         EnglishProductName
'''

In [108]:
salessummarydf = aw_sql_sqlsec.get_db_data(sql)
salessummarydf

,Category,SubCat,Product,SalesAmount
0,Accessories,Bike Racks,Hitch Rack - 4-Bike,39360.00
1,Accessories,Bike Stands,All-Purpose Bike Stand,39591.00
2,Accessories,Bottles and Cages,Mountain Bottle Cage,20229.75
3,Accessories,Bottles and Cages,Road Bottle Cage,15390.88
4,Accessories,Bottles and Cages,Water Bottle - 30 oz.,21177.56
...,...,...,...,...
125,Clothing,Socks,"Racing Socks, L",2427.30
126,Clothing,Socks,"Racing Socks, M",2679.02
127,Clothing,Vests,"Classic Vest, L",12382.50
128,Clothing,Vests,"Classic Vest, M",12636.50


In [109]:
# Store the results in our SQLite data warehouse

# if_exists options are replace, append, 
salessummarydf.to_sql('internetsalessummary', con=slconn, index=False, if_exists='replace')

130

**Getting meta data...**

In [110]:
salessummarydf.columns 

Index(['Category', 'SubCat', 'Product', 'SalesAmount'], dtype='object')

In [111]:
salessummarydf.dtypes

Category        object
SubCat          object
Product         object
SalesAmount    float64
dtype: object

In [112]:
# Get table schema...
import pandas as pd 

pd.read_sql_query("""
PRAGMA table_info('internetsalessummary');
""", slconn)

,cid,name,type,notnull,dflt_value,pk
0,0,Category,TEXT,0,None,0
1,1,SubCat,TEXT,0,None,0
2,2,Product,TEXT,0,None,0
3,3,SalesAmount,REAL,0,None,0


### Querying data in SQLite...

In [113]:
import sqlite3
import pandas as pd

pd.read_sql_query("SELECT * FROM internetsalessummary", slconn).head(4)

,Category,SubCat,Product,SalesAmount
0,Accessories,Bike Racks,Hitch Rack - 4-Bike,39360.00
1,Accessories,Bike Stands,All-Purpose Bike Stand,39591.00
2,Accessories,Bottles and Cages,Mountain Bottle Cage,20229.75
3,Accessories,Bottles and Cages,Road Bottle Cage,15390.88


In [114]:
#  Note the round(2) function to elimiate scientific notation in pandas...

#  index_col=['Category'] to eliminate the df index 

pd.read_sql_query('''
SELECT Category, sum(SalesAmount) as Sales  
FROM internetsalessummary
GROUP BY Category''', slconn, index_col=['Category']).round(2)

,Sales
Category,
Accessories,700759.96
Bikes,28318144.65
Clothing,339772.61


We want to build some dimension tables but let's think about it first...

In [115]:
# We just need to pass in the query string.  The connection properties are already set.
aw_sql_sqlsec.get_db_data('''
SELECT * 
FROM dimproductcategory''').head(2)

,ProductCategoryKey,ProductCategoryAlternateKey,EnglishProductCategoryName,SpanishProductCategoryName,FrenchProductCategoryName
0,1,1,Bikes,Bicicleta,Vélo
1,2,2,Components,Componente,Composant


In [116]:
aw_sql_sqlsec.get_db_data('''
SELECT ProductKey, ProductSubCategoryKey, EnglishProductName as Name, Status 
FROM dimproduct''').head(2)

,ProductKey,ProductSubCategoryKey,Name,Status
0,1,NaN,Adjustable Race,Current
1,2,NaN,Bearing Ball,Current


Let's download the Product Dimension tables as a single SQLite table.

In [117]:
sql = '''
SELECT p.ProductKey, s.ProductSubcategoryKey, c.ProductCategoryKey, 
EnglishProductCategoryName as Category, 
EnglishProductSubcategoryName as Subcategory, ModelName as Model
FROM       dimproduct                   p
INNER JOIN dimproductsubcategory        s
ON (p.ProductSubcategoryKey = s.ProductSubcategoryKey)
INNER JOIN dimproductcategory           c
ON (s.ProductCategoryKey = c.ProductCategoryKey)
WHERE p.Status = 'Current' OR p.Status = 'NULL' 
'''

In [118]:
aw_sql_sqlsec.get_db_data(sql).head(3)

,ProductKey,ProductSubcategoryKey,ProductCategoryKey,Category,Subcategory,Model
0,210,14,2,Components,Road Frames,HL Road Frame
1,211,14,2,Components,Road Frames,HL Road Frame
2,214,31,4,Accessories,Helmets,Sport-100


the to_sql() dataframe method will save the data to our SQLite database.

In [119]:
# if_exists has 3 options: replace, append, fail

aw_sql_sqlsec.get_db_data(sql).to_sql('dimproduct', con=slconn, index=False, if_exists='replace')

197

In [120]:
# Let's confirm the table is there...
import sqlite3
import pandas as pd

pd.read_sql_query("SELECT * FROM dimproduct limit 3", slconn).head(3)

,ProductKey,ProductSubcategoryKey,ProductCategoryKey,Category,Subcategory,Model
0,210,14,2,Components,Road Frames,HL Road Frame
1,211,14,2,Components,Road Frames,HL Road Frame
2,214,31,4,Accessories,Helmets,Sport-100


What other data do we want?

In [121]:
aw_sql_sqlsec.get_db_data('''select top 1 * from factinternetsales''')

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,...,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.27,286.2616,89.4568,None,None,2010-12-29,2011-01-10,2011-01-05


In [122]:
aw_sql_sqlsec.get_db_data('select * from factinternetsales'). \
to_sql('factinternetsales', con=slconn, index=False, if_exists='replace')

60398

In [123]:
pd.read_sql_query("SELECT * FROM factinternetsales limit 3", slconn)

,ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,...,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,CarrierTrackingNumber,CustomerPONumber,OrderDate,DueDate,ShipDate
0,310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,...,2171.2942,2171.2942,3578.27,286.2616,89.4568,None,None,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
1,346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,...,1912.1544,1912.1544,3399.99,271.9992,84.9998,None,None,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00
2,346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,...,1912.1544,1912.1544,3399.99,271.9992,84.9998,None,None,2010-12-29 00:00:00,2011-01-10 00:00:00,2011-01-05 00:00:00


In [124]:
aw_sql_sqlsec.get_db_data('select * from dimcustomer'). \
to_sql('dimcustomer', con=slconn, index=False, if_exists='replace')

18484

In [125]:
aw_sql_sqlsec.get_db_data('select * from dimdate'). \
to_sql('dimdate', con=slconn, index=False, if_exists='replace')

3652

### Create a SQLite view that pulls all the data together...
Note the 'if not exists' - so you don't get an error on reruns...

In [126]:
sql = '''
CREATE VIEW IF NOT EXISTS vsalesinfo as  
SELECT  OrderDateKey, DueDateKey, s.CustomerKey, SalesTerritoryKey,
SalesAmount, p.*, Gender, YearlyIncome as Salary, OrderQuantity, TotalProductCost, TaxAmt, 
CASE WHEN NumberChildrenAtHome > 0 THEN 'Y' ELSE 'N' END as HasChildren, 
c.EnglishEducation as Education, c.CommuteDistance,
c.BirthDate, 
CAST( ((JulianDay('now')) - JulianDay(BirthDate))/365.2 as int) as Age,
d.FiscalYear, d.FiscalQuarter, d.EnglishMonthName as Month, MonthNumberOfYear, d.CalendarYear
FROM factinternetsales                s
INNER JOIN dimcustomer                c
ON (s.CustomerKey = c.CustomerKey)
INNER JOIN dimproduct                 p
ON (s.ProductKey = p.ProductKey)
INNER JOIN dimdate                    d
ON (s.OrderDateKey = d.DateKey);
'''

In [127]:
# We created the cursor to the SQLite database at the beginning.

cursor.execute(sql)

In [128]:
pd.read_sql_query('select * from vsalesinfo', slconn).head(3)

,OrderDateKey,DueDateKey,CustomerKey,SalesTerritoryKey,SalesAmount,ProductKey,ProductSubcategoryKey,ProductCategoryKey,Category,Subcategory,...,HasChildren,Education,CommuteDistance,BirthDate,Age,FiscalYear,FiscalQuarter,Month,MonthNumberOfYear,CalendarYear
0,20121228,20130109,18239,9,2443.35,380,2,1,Bikes,Road Bikes,...,N,Graduate Degree,0-1 Miles,1980-07-19,42,2012,2,December,12,2012
1,20121228,20130109,18239,9,8.99,479,28,4,Accessories,Bottles and Cages,...,N,Graduate Degree,0-1 Miles,1980-07-19,42,2012,2,December,12,2012
2,20121228,20130109,27873,9,1214.85,577,3,1,Bikes,Touring Bikes,...,N,Bachelors,0-1 Miles,1983-10-01,39,2012,2,December,12,2012


### Load CSV file into our SQLite database...

In [129]:
salesterritory = pd.read_csv('.\data\dimsalesterritory.csv')  
salesterritory.head(2)

,SalesTerritoryKey,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup,SalesTerritoryImage
0,1,1,Northwest,United States,North America,0xFFD8FFE000104A46494600010101004800480000FFDB...
1,2,2,Northeast,United States,North America,0xFFD8FFE000104A46494600010101004800480000FFDB...


In [130]:
salesterritory.to_sql('dimsalesterritory', con=slconn, index=False, if_exists='replace')

11

In [131]:
sql = '''
SELECT t.SalesTerritoryCountry, t.SalesTerritoryRegion, SUM(SalesAmount)
FROM  FactInternetSales   s
JOIN  DimSalesTerritory   t
ON (s.SalesTerritoryKey = t.SalesTerritoryKey)
GROUP BY SalesTerritoryCountry, SalesTerritoryRegion
ORDER BY SUM(SalesAmount) DESC
'''

In [132]:
pd.read_sql_query(sql, slconn, index_col=['SalesTerritoryCountry']).round(2)

,SalesTerritoryRegion,SUM(SalesAmount)
SalesTerritoryCountry,,
Australia,Australia,9061000.58
United States,Southwest,5718150.81
United States,Northwest,3649866.55
United Kingdom,United Kingdom,3391712.21
Germany,Germany,2894312.34
France,France,2644017.71
Canada,Canada,1977844.86
United States,Southeast,12238.85
United States,Northeast,6532.47


### Let's bring in some data from a PostgreSQL database...

In [133]:
actors_pg_df = aw_postgresdb.get_db_data('select * from customers')

In [134]:
actors_pg_df.to_sql('customers', con=slconn, index=False, if_exists='replace')

91

In [135]:
pd.read_sql_query('select * from customers limit 3', slconn)

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None


### We can drop the table since we are done.

In [136]:
cursor.execute('''select * from internetsalessummary limit 5;''').fetchall()

[('Accessories', 'Bike Racks', 'Hitch Rack - 4-Bike', 39360.0),
 ('Accessories', 'Bike Stands', 'All-Purpose Bike Stand', 39591.0),
 ('Accessories', 'Bottles and Cages', 'Mountain Bottle Cage', 20229.75),
 ('Accessories', 'Bottles and Cages', 'Road Bottle Cage', 15390.88),
 ('Accessories', 'Bottles and Cages', 'Water Bottle - 30 oz.', 21177.56)]

In [137]:
cursor.execute('''DROP TABLE internetsalessummary''')
slconn.commit()

In [138]:
cursor.execute('''select * from internetsalessummary;''').fetchall()

OperationalError: no such table: internetsalessummary

**Let's close the connection**

In [139]:
cursor.close()
slconn.close()